In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import json

# Detectron colors
_COLORS = np.array([
    0.000, 0.447, 0.741,
    0.850, 0.325, 0.098,
    0.929, 0.694, 0.125,
    0.494, 0.184, 0.556,
    0.466, 0.674, 0.188
]).astype(np.float32).reshape((-1, 3))

# Random number generator seed
_RNG_SEED = 1

# Fix RNG seeds
random.seed(_RNG_SEED)
np.random.seed(_RNG_SEED)

# Directory where sweep summaries are stored
_DATA_DIR = '../data'

# Max flops constraint
_MAX_F = 0.129
# Max params constraint
_MAX_P = 0.856

In [ ]:
def load_sweep(sweep_name):
    """Loads a sweep summary."""
    summary_path = os.path.join(_DATA_DIR, '{}.json'.format(sweep_name))
    with open(summary_path, 'r') as f:
        sweep_summary = json.load(f)
    return sweep_summary

In [ ]:
def is_valid_p(job, min_p, max_p):
    return min_p < job['params'] * 1e-6 and job['params'] * 1e-6 < max_p

def is_valid_f(job, min_f, max_f):
    return min_f < job['flops'] * 1e-9 and job['flops'] * 1e-9 < max_f

In [ ]:
def compute_norm_ws(cs, num_bins, c_range):
    """Computes normalized EDF weights."""
    hist, edges = np.histogram(cs, bins=num_bins, range=c_range)
    inds = np.digitize(cs, bins=edges) - 1
    assert np.count_nonzero(hist) == num_bins
    return 1 / hist[inds] / num_bins

In [ ]:
# ResNeXt sweeps
sweeps = {
    'ResNeXt-A': load_sweep('ResNeXt-A'),
    'ResNeXt-B': load_sweep('ResNeXt-B')
}

In [ ]:
print('Figure 4\n')

dss = ['ResNeXt-A', 'ResNeXt-B']
cms = ['params', 'flops']
k = 6000

random.seed(_RNG_SEED)
r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

for i, cm in enumerate(cms):
    ax = axes[i]
    for j, ds in enumerate(dss):
        if cm == 'params':
            sweep = [job for job in sweeps[ds] if is_valid_p(job, 0, _MAX_P)]
            sweep = random.sample(sweep, min(len(sweep), k))
            xs = [job['params'] * 1e-6 for job in sweep]
            ys = [job['min_test_top1'] for job in sweep]
            ax.set_xlabel('params (M)', fontsize=16)
            ax.set_xlim([0, _MAX_P])
        if cm == 'flops':
            sweep = [job for job in sweeps[ds] if is_valid_f(job, 0, _MAX_F)]
            sweep = random.sample(sweep, min(len(sweep), k))
            xs = [job['flops'] * 1e-9 for job in sweep]
            ys = [job['min_test_top1'] for job in sweep]
            ax.set_xlabel('flops (B)', fontsize=16)
            ax.set_xlim([0, _MAX_F])
        ax.scatter(xs, ys, color=_COLORS[j], alpha=0.05, label=ds, rasterized=True)
        ax.grid(alpha=0.4)
        ax.set_ylabel('error', fontsize=16)
        ax.set_ylim([4, 20])
        ax.legend(loc='upper right', prop={'size' : 13})

plt.tight_layout();

In [ ]:
print('Figure 5\n')

dss = ['ResNeXt-A', 'ResNeXt-B']
cms = ['params', 'flops']

r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

for i, cm in enumerate(cms):
    ax = axes[i]
    for j, ds in enumerate(dss):
        if cm == 'params':
            vs = [job['params'] * 1e-6 for job in sweeps[ds] if is_valid_p(job, 0, _MAX_P)]
            ax.set_xlabel('params (M)', fontsize=16)
            ax.set_xlim([0, _MAX_P])
            c_range = (0, _MAX_P)
        if cm == 'flops':
            vs = [job['flops'] * 1e-9 for job in sweeps[ds] if is_valid_f(job, 0, _MAX_F)]
            ax.set_xlabel('flops (B)', fontsize=16)
            ax.set_xlim([0, _MAX_F])
            c_range = (0, _MAX_F)
        ax.hist(vs, bins=36, range=c_range, color=_COLORS[j], alpha=0.6, label=ds)
        ax.grid(alpha=0.4)
        ax.set_ylabel('number of models', fontsize=16)
        ax.set_ylim([0, 2000])
        ax.legend(loc='upper right', prop={'size': 13})

plt.tight_layout();

In [ ]:
print('Figure 3\n')

num_bins = 40
dss = ['ResNeXt-A', 'ResNeXt-B']
cms = ['none', 'params', 'flops']

r, c = 1, 3
w, h = 3, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

min_p, max_p = 0.02, _MAX_P
min_f, max_f = 0.008, _MAX_F

for i, cm in enumerate(cms):
    ax = axes[i]
    for j, ds in enumerate(dss):
        if cm == 'none':
            errs = [
                job['min_test_top1'] for job in sweeps[ds]
                if is_valid_p(job, 0, _MAX_P) and is_valid_f(job, 0, max_f)
            ]
            errs = sorted(errs)
            ws = np.ones(len(errs)) / len(errs)
            ax.set_xlabel('error', fontsize=16)
        if cm == 'params':
            sweep = [job for job in sweeps[ds] if is_valid_p(job, min_p, max_p)]
            errs = np.array([job['min_test_top1'] for job in sweep])
            ps = np.array([job['params'] * 1e-6 for job in sweep])
            inds = np.argsort(errs)
            errs, ps = errs[inds], ps[inds]
            ws = compute_norm_ws(ps, num_bins, c_range=(min_p, max_p))
            ax.set_xlabel('error | params', fontsize=16)
        if cm == 'flops':
            sweep = [job for job in sweeps[ds] if is_valid_f(job, min_f, max_f)]
            errs = np.array([job['min_test_top1'] for job in sweep])
            fs = np.array([job['flops'] * 1e-9 for job in sweep])
            inds = np.argsort(errs)
            errs, fs = errs[inds], fs[inds]
            ws = compute_norm_ws(fs, num_bins, c_range=(min_f, max_f))
            ax.set_xlabel('error | flops', fontsize=16)
        assert np.isclose(np.sum(ws), 1.0)
        ax.plot(
            errs, np.cumsum(ws),
            color=_COLORS[j], linewidth=2, alpha=0.8, label=ds
        )
    ax.grid(alpha=0.4)
    ax.set_ylabel('cumulative prob.', fontsize=16)
    ax.set_xlim([4.5, 17.5])
    ax.legend(loc='lower right', prop={'size': 13})

plt.tight_layout();

In [ ]:
def bin_jobs(jobs, key, num_bins, min_val=None, max_val=None):
    """Groups jobs into bins based on key value."""
    # Compute the lower range
    if min_val is None:
        min_val = min([job[key] for job in jobs])
    # Compute the upper range
    if max_val is None:
        max_val = max([job[key] for job in jobs])
    # Compute the bin size
    bin_size = (max_val - min_val) / num_bins
    # Split jobs into bins
    bins = [[] for i in range(num_bins)]
    for job in jobs:
        # Skip values outside the desired range
        if job[key] < min_val or max_val < job[key]:
            continue
        # Compute the bin index
        bin_ind = int((job[key] - min_val) / bin_size)
        # Subtract one for values that are exactly on the upper range
        bin_ind =  bin_ind if bin_ind != num_bins else bin_ind - 1
        # Assign the job to a bin
        bins[bin_ind].append(job)
    return bins

In [ ]:
print('Figure 15\n')

r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

dss = ['ResNeXt-A', 'ResNeXt-B']
cms = ['params', 'flops']
num_bins = 20

for i, cm in enumerate(cms):
    # Bin jobs by complexity
    job_bins = {}
    for j, ds in enumerate(dss):
        sweep = sweeps[ds]
        if cm == 'params':
            min_val, max_val = 0, _MAX_P * 1e6
        if cm == 'flops':
            min_val, max_val = 0, _MAX_F * 1e9
        job_bins[ds] = bin_jobs(sweep, cm, num_bins, min_val, max_val)
    # Plot dist per bin
    ax = axes[i]
    for j, ds in enumerate(dss):
        # Compute stats per bin
        bin_inds = []
        mus, stds = [], []
        for bin_ind in range(num_bins):
            errs = [job['min_test_top1'] for job in job_bins[ds][bin_ind]]
            if not errs:
                continue
            bin_inds.append(bin_ind)
            mus.append(np.mean(errs))
            stds.append(np.std(errs))
        mus, stds = np.array(mus), np.array(stds)
        ax.scatter(bin_inds, mus, color=_COLORS[j], alpha=0.6, label=ds)
        ax.fill_between(bin_inds, mus - 2 * stds, mus + 2 * stds, color=_COLORS[j], alpha=0.1)
    ax.set_xlim([0, num_bins])
    ax.set_ylim([4, 20])
    ax.set_ylabel('error', fontsize=16)
    ax.set_xlabel('{} bucket'.format(cm), fontsize=16)
    ax.legend(loc='upper right', fontsize=13, ncol=1)
    ax.grid(alpha=0.4)

plt.tight_layout();